<a href="https://colab.research.google.com/github/jesusGS01/BDGE/blob/main/P1_TGINE_JesusGarciaSalmeron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 1 : Extracción y búsqueda de información textual

Alumno: Jesús García Salmerón

Convocatoria: Enero, 2024

## Parte 1- Compilar datos de documentos web

In [1]:
!pip3 install -U scrapy

In [2]:
import scrapy
import sys
import json
import locale
import time
import random
from bs4 import BeautifulSoup

# Para cada crawler que nos definimos nos debemos crear una clase Spider que debe heredar de la clase scrapy.Spider

class UMSpider (scrapy.Spider):
    name = 'animals_nationalGeographic'

    # Decimos que el dominio válido es el de la UM
    allowed_domains = ['www.nationalgeographic.com.es']

    # podemos definir las páginas de inicio
    start_urls = ['https://www.nationalgeographic.com.es/mundo-animal/tiburones-sobrevivieron-casi-20-anos-lago-campo-golf-como-hicieron_20748']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # debemos de implementar este método que se llamará para cada una de las páginas que se vayan analizando
    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        # Guardamos la URL del sitio que se está visitando
        url = str(response.request.url).strip()

        # Cogemos el contenido relevante y para eso debemos usar selectores CSS
        for article in response.css ('.article-main'):
            # Cogemos el contenido del título
            title = str (article.css ('h1::text').get ()).strip()
            title = BeautifulSoup (title, 'html.parser').get_text().strip()
            subtitle = str (article.css ('h2::text').get ()).strip()
            subtitle = BeautifulSoup (subtitle, 'html.parser').get_text().strip()

            # Cogemos el contenido del contenido
            content = ''.join (article.css('.article-text.txt').get())
            soup  = BeautifulSoup (content, 'html.parser')

            # Lista de clases a eliminar
            unwanted_tags = ["subscription-highlighted", "related-article-content", "ad-inline"]

            # Buscamos y eliminamos las etiquetas no deseadas
            for tag in unwanted_tags:
                for el in soup.find_all(class_=tag):
                    el.extract()

            content = soup.get_text().strip().replace("\"","")

            # Cogemos también la fecha si existe
            date = str(article.css ('.article-top-info time').get().strip())
            if (date):
                date = BeautifulSoup (date, 'html.parser').get_text().strip()

            print ("-------------------------")
            print (url)
            print (title)
            print (subtitle)
            print (content)
            print (date)
            print ("-------------------------")

            data = {
                'url' : url,
                'title': title,
                'subtitle': subtitle
                #'content': content,
                # 'date': date
            }

            # filename = str(random.random()).replace(".","") + ".json"

            # # Guardamos el documento si tiene contenido y título
            # if content and title:
            #     with open ('animals/' + filename, 'w') as f:
            #         json.dump (data, f, indent = 4)


        # Obtenemos todas las otros links de la página representados por la etiqueta <a>
        # url_in_current_document = response.css ('a')
        # for next_page in url_in_current_document:
        #     # Para limitar que solamente se parseen las noticias dentro de 'https://www.um.es/web/sala-prensa/'
        #     url_str = str(next_page.css('::attr(href)').get())
        #     if "www.um.es/web/sala-prensa/" in url_str:
        #       # obtenemos el atributo href de la etiqueta <a> y parseamos la página
        #       yield response.follow (next_page, self.parse)

In [3]:
import scrapy, os
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (os.path.exists('animals')== False):
    os.mkdir('animals')

# Creamos el proceso con el RSSSpider
process.crawl(UMSpider)
# Ejecutamos el Crawler
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
DEBUG:scrap

-------------------------
https://www.nationalgeographic.com.es/mundo-animal/tiburones-sobrevivieron-casi-20-anos-lago-campo-golf-como-hicieron_20748
Tiburones sobrevivieron casi 20 años en lago de campo de golf, ¿cómo lo hicieron?
Las grandes inundaciones provocadas por la crecida de los ríos Logan y Albert habían arrastrado hasta allí a estos escualos, que no pudieron salir del lago una vez las aguas retrocedieron.
En 1996, seis tiburones sarda (Carcharhinus leucas) quedaron atrapados en las aguas de un lago de un campo de golf cerca de Brisbane, en Australia. Las grandes inundaciones provocadas por la crecida de los ríos Logan y Albert habían arrastrado hasta allí a estos escualos, que no pudieron salir del lago una vez las aguas retrocedieron. Allí estuvieron durante casi dos décadas, sobreviviendo gracias al cuidado del personal del parque.Aunque aquel acontecimiento tuvo un gran eco en los medios de comunicación, sus consecuencias biológicas nunca se habían investigado con profun